## Generating and Visualizing Antipodal Grasps on Book

In [1]:
import numpy as np
import trimesh
from typing import List 
from IPython.display import clear_output
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    Concatenate,
    DiagramBuilder,
    MeshcatVisualizer,
    MeshcatVisualizerParams,
    Parser,
    PointCloud,
    RigidTransform,
    StartMeshcat,
    UniformlyRandomRotationMatrix,
    Context,
    Diagram,
    PointCloud
)
from pydrake.geometry import Meshcat

from manipulation import running_as_notebook
from manipulation.scenarios import AddFloatingRpyJoint, AddRgbdSensors, ycb
from manipulation.utils import ConfigureParser
from manipulation.clutter import GraspCandidateCost, GenerateAntipodalGraspCandidate
from manipulation.icp import IterativeClosestPoint

# Own utils
from hwstation.utils import setup_builder, plot_and_simulate
from hwstation.add_objects import get_library_scenario_data, get_library_scenario_data_without_robot

Starting Meshcat

In [2]:
# Start meshcat
try:
    meshcat = Meshcat(7006)
except:
    pass #This error appears if this cell is executed twice (port 7006 is already taken then)

INFO:drake:Meshcat listening for connections at http://localhost:7006


Pointcloud Related Functions (to be used when finding point cloud to do grasp)

In [3]:
def get_table_pointclouds(diagram_context: Context, diagram: Diagram):
    point_cloud_dict = {}
    for idx in range(4):
        point_cloud_dict[f"table_camera_{idx}_ptcloud"] = diagram.GetOutputPort(f"table_camera_{idx}_ptcloud").Eval(diagram_context)
    return point_cloud_dict


def merge_point_clouds(table_pointclouds: dict, 
                        downsample_factor: float = 0.005,
                        lower_xyz: List[float] = [0.0, -0.2, 0.5564], 
                        upper_xyz: List[float] = [1.5, 1.5, 0.8]):
    pcd = []
    for key in table_pointclouds.keys():
        cloud = table_pointclouds[key]
        pcd.append(
            cloud.Crop(lower_xyz=lower_xyz, upper_xyz=upper_xyz)
            )
    merged_pcd = Concatenate(pcd)
    down_sampled_pcd = merged_pcd.VoxelizedDownSample(voxel_size=0.005)
    return down_sampled_pcd

def get_merged_pointcloud(diagram_context: Context, diagram: Diagram):
    
    #Get merged point cloud from all cameras:
    table_pointclouds = get_table_pointclouds(diagram_context, diagram)
    merged_pcd = merge_point_clouds(table_pointclouds)

    #Ensure that all number are finite:
    merged_pcd_np = merged_pcd.xyzs().transpose()
    mask_points = (merged_pcd_np== np.inf).all(axis=1)
    if mask_points.any():
        sys.exit("No infinite points were expected")
    return merged_pcd

def convert_obj_to_pc(filename: str, n_samples: int = 10000, show: bool =False) -> np.ndarray:
    book_mesh = trimesh.load(filename)
    book_hull = book_mesh.convex_hull
    sample_points = book_hull.sample(n_samples)
    point_cloud = trimesh.points.PointCloud(sample_points)
    if show:
        scene = trimesh.Scene([book_hull, point_cloud])
        scene.show()
    return np.array(point_cloud.vertices).transpose()

Environment Generation Functions

In [4]:
# Function to make environment and return the diagram and context
def make_environment_model():

    # Get string describing scenario
    scenario_data = get_library_scenario_data()

    #Setting up all drake simulation objects:
    builder, plant, scene_graph, station, parser, scenario = setup_builder(meshcat, scenario_data=scenario_data)

    # Allowing the Parser to access all of the aspects in the environment including the robot (iiwa)
    parser.AddModelsFromUrl("file:///workspaces/RobotLibrarian/hwstation/objects/library_setup.dmd.yaml")

    #Simulate environment (right now, only the books fall on the table)
    diagram, plant_context, simulator = plot_and_simulate(meshcat, builder, plant, station, time_end=0.0)

    context = diagram.CreateDefaultContext()

    return diagram, context, plant

# Another diagram for the objects the robot "knows about": shelves, table, cameras, books (only one for now), and a gripper.  Think of this as the model in the robot's head.
def make_internal_model():
    # Get string describing scenario
    scenario_data = get_library_scenario_data_without_robot()

    #Setting up all drake simulation objects:
    builder, plant, scene_graph, station, parser, scenario = setup_builder(meshcat, scenario_data=scenario_data)

    # This parser has all items in the environment except the robot and instead has a "floating" gripper to try and find antipodal grasps with
    parser.AddModelsFromUrl("file:///workspaces/RobotLibrarian/hwstation/objects/library_setup_floating_gripper.dmd.yaml")
    plant.Finalize()
    return builder.Build()

Finding and Visualizing the Gripper Grasp Poses

In [5]:
# Function just to draw the gripper at the pose that was found
def draw_grasp_candidate(X_G, prefix="gripper", draw_frames=True):
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
    parser = Parser(plant)
    ConfigureParser(parser)
    parser.AddModelsFromUrl(
        "package://manipulation/schunk_wsg_50_welded_fingers.sdf"
    )
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("body"), X_G)
    plant.Finalize()

    # frames_to_draw = {"gripper": {"body"}} if draw_frames else {}
    params = MeshcatVisualizerParams()
    params.prefix = prefix
    params.delete_prefix_on_initialization_event = False
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat, params
    )
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    diagram.ForcedPublish(context)
 

# The main process for finding the grasp poses and visualizing them
def sample_grasps_example():
    meshcat.Delete()
    rng = np.random.default_rng()

    # Generating the environment we are working in
    environment, environment_context, environment_plant = make_environment_model()

    # Generating the internal model we will be working with to find grasp poses
    internal_model = make_internal_model()
    internal_model_context = internal_model.CreateDefaultContext()

    # Finally a model for the visualization of grasp poses itself.
    # Get string describing scenario
    scenario_data = get_library_scenario_data()

    #Setting up all drake simulation objects:
    builder, plant, scene_graph, station, parser, scenario = setup_builder(meshcat, scenario_data=scenario_data)

    plant.Finalize()

    # Code from the grasp_selection Drake notebook
    params = MeshcatVisualizerParams()
    params.prefix = "planning"
    visualizer = MeshcatVisualizer.AddToBuilder(
        builder, scene_graph, meshcat, params
    )
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    diagram.ForcedPublish(context)

    # Hide the planning gripper
    meshcat.SetProperty("planning/gripper", "visible", False)

    # Point clouds to obtain the cloud we will work with for grasps:
    scene_pcl = get_merged_pointcloud(internal_model_context, internal_model)
    meshcat.SetObject("merged_cropped_pcl", cloud=scene_pcl, point_size=0.004)

    book_filename = "hwstation/objects/book.obj"
    model_pcl = convert_obj_to_pc(book_filename, show=False)

    height_table = 0.5
    initial_guess = RigidTransform(p=[0.0,0.0,height_table])

    X_MS_hat, chat = IterativeClosestPoint(
        p_Om=model_pcl,
        p_Ws=scene_pcl.xyzs(),
        X_Ohat=initial_guess,
        meshcat=meshcat,
        meshcat_scene_path="icp",
        max_iterations=25,
    )

    transformed_model_pcl = X_MS_hat @ model_pcl

    cloud = PointCloud(transformed_model_pcl.shape[1])
    cloud.mutable_xyzs()[:] = transformed_model_pcl
    cloud.EstimateNormals(radius=0.5, num_closest=50)

    meshcat.SetObject("planning/cloud", cloud, point_size=0.003)

    plant.GetMyContextFromRoot(context)
    scene_graph.GetMyContextFromRoot(context)

    # Now find grasp poses
    # X_Gs will have the poses to be used for planning when working on that step
    costs = []
    X_Gs = []
    for i in range(100 if running_as_notebook else 2):
        cost, X_G = GenerateAntipodalGraspCandidate(
            internal_model, internal_model_context, cloud, rng
        )
        if np.isfinite(cost):
            costs.append(cost)
            X_Gs.append(X_G)

    indices = np.asarray(costs).argsort()[:5]
    min_cost_XGs = []
    for idx in indices:
        min_cost_XGs.append(X_Gs[idx])

    #Get the antipodal grasp that is closest to the robot WSG frame:
    positions = np.stack([frame.translation() for frame in min_cost_XGs])

    plant_context = environment_plant.CreateDefaultContext()

    wsg = environment_plant.GetBodyByName("body")
    wsg_body_index = wsg.index()

    wsg_pose = environment_plant.EvalBodyPoseInWorld(plant_context,wsg)
    wsg_pos = wsg_pose.translation()

    best_grasp_idx = np.argmin(((positions-wsg_pos)**2).sum(axis=1))
    X_G_optim = min_cost_XGs[best_grasp_idx]

    #for rank, index in enumerate(indices):
    draw_grasp_candidate(
        X_G_optim, prefix=f"best grasp", draw_frames=False
    )

    return X_G_optim


optim_frame = sample_grasps_example()

fixing improper rotation
